# 2023-12-07

In [1]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=7)

## Initial solution
In transit due to business trip, started at (UTC-5:35) (T+35 minutes)
```
      --------Part 1--------   --------Part 2--------
Day       Time   Rank  Score       Time   Rank  Score
  7   01:49:04  11003      0   02:52:36  11162      0
```

### Puzzle 1 & 2

In [3]:
import numpy as np
from collections import Counter
from functools import cmp_to_key

hands = [
    "five_of_a_kind",
    "four_of_a_kind",
    "full_house",
    "three_of_a_kind",
    "two_pairs",
    "one_pair",
    "high_card",
]
mapping = {
    5: "five_of_a_kind",
    4: "four_of_a_kind",
    (3, 2): "full_house",
    3: "three_of_a_kind",
    (2, 2): "two_pairs",
    2: "one_pair",
    1: "high_card",
}
cards_map = {"A": 14, "K": 13, "Q": 12, "J": 11, "T": 10}
cards_map_p2 = {"A": 14, "K": 13, "Q": 12, "J": 1, "T": 10}


class Hand:
    @staticmethod
    def score(cards, mode="p1"):
        counts = Counter(cards)
        most_common = counts.most_common(1)
        if most_common[0][1] == 5:
            return mapping[5]
        if mode == "p2":
            while "J" in cards:
                most_freq = [x[1] for x in counts.most_common() if x[0] != "J"][0]
                top_candidates = [
                    (key, val)
                    for key, val in counts.items()
                    if val == most_freq and key != "J"
                ]
                if top_candidates == []:
                    return mapping[5]
                card_to_add = top_candidates[0][0]
                cards = cards.replace("J", card_to_add)
            counts = Counter(cards)
        if counts.most_common(1)[0][1] == 5:
            return mapping[5]
        try:
            most_common, second_most_common = counts.most_common(2)
        except ValueError:
            print(counts)
        if second_most_common[1] == 2:
            return mapping[(most_common[1], second_most_common[1])]
        else:
            return mapping[most_common[1]]

    @staticmethod
    def calculate_rank(hand_type):
        return 5 - hands.index(hand_type)

    def __init__(self, cards, bid=0, mode="p1") -> None:
        self.cards = cards
        self.hand = self.score(cards, mode=mode)
        self.rank = self.calculate_rank(self.hand)
        self.bid = int(bid)
        card_val_mapping = cards_map if mode == "p1" else cards_map_p2
        self.cards_val = [int(card_val_mapping.get(x, x)) for x in cards]


def comperator(hand1: Hand, hand2: Hand):
    if hand1.rank > hand2.rank:
        return 1
    elif hand1.rank < hand2.rank:
        return -1
    elif hand1.cards_val < hand2.cards_val:
        return -1
    return 1


games = np.array([x.split(" ") for x in puzzle.input_data.splitlines()])
player_hands = list(map(lambda x, y: Hand(x, y), games[:, 0], games[:, 1]))
player_hands.sort(key=cmp_to_key(comperator), reverse=True)
puzzle.answer_a = sum(
    [(len(player_hands) - i) * x.bid for i, x in enumerate(player_hands)]
)

player_hands = list(map(lambda x, y: Hand(x, y, mode="p2"), games[:, 0], games[:, 1]))
player_hands.sort(key=cmp_to_key(comperator), reverse=True)
puzzle.answer_b = sum(
    [(len(player_hands) - i) * x.bid for i, x in enumerate(player_hands)]
)